## Get orders for test shopper

In [1]:
df = spark.sql("select * from tecton_dev.ads_bill_line limit 10")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1660436778279_0010,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+-----------------+--------------+----------------+--------------------+--------------------+----------------+----------------+---------------+-------------------------+-------------------------+----------------------------+---------------------------+------------------------------+---------------------------------+-------------------------+-------------------+--------------------+-------------------+-------------------+------------------+----------------------------+-----------------------+---------------------+-------------------------+--------------------+---------------------+------------------------------+------------------+-------------+-------------------+---------------------+---------------------+---------------+----------------------+----------------------+----------------------------+-----------------+-------------------------+-------------------------+-------------------------+------------------------------+---------------------------+-------------

In [4]:
# required to enable explain plan
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
orders = spark.sql("""
select
  merged_shopper_id,
  bill_id,
  bill_modified_mst_ts,
  product_pnl_line_name,
  unit_qty,
  receipt_price_usd_amt
from tecton_dev.ads_bill_line 
where merged_shopper_id = '292787210'
order by bill_modified_mst_ts desc
""")
orders.show(truncate=False)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1659375682136_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+-----------------------+------------------------+--------+---------------------+
|merged_shopper_id|bill_id   |bill_modified_mst_ts   |product_pnl_line_name   |unit_qty|receipt_price_usd_amt|
+-----------------+----------+-----------------------+------------------------+--------+---------------------+
|292787210        |2109454345|2022-05-24 06:40:40    |Virtual Hosting         |1       |102.54               |
|292787210        |2109454345|2022-05-24 06:40:40    |Virtual Hosting         |1       |11.60                |
|292787210        |2097138481|2022-04-24 20:58:05    |Virtual Hosting         |1       |104.13               |
|292787210        |2097138481|2022-04-24 20:58:05    |Virtual Hosting         |1       |11.78                |
|292787210        |2064633178|2022-03-16 23:32:57    |Virtual Hosting         |1       |11.78                |
|292787210        |2064633178|2022-03-16 23:32:57    |Virtual Hosting         |1       |104.13               |
|

## Test customer_last_order_90d

### run

In [2]:
import tecton
from datetime import date, datetime, timedelta

ws = tecton.get_workspace('jfarr')
fv = ws.get_feature_view('customer_last_order_90d')

df = fv.run(start_time=datetime(2022, 1, 1), end_time=datetime.now()).to_spark()
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1792

In [3]:
df2 = df.filter(df.shopper_id == '292787210')
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-----------------------+----------------------------+-------------+-------------------------+----------------------+-----------------+------------------+
|shopper_id|      window_end_ts|crm_portfolio_type_name|c3_call_center_location_name|reseller_name|primary_payment_type_name|point_of_purchase_name|bill_sub_geo_code|purchase_path_name|
+----------+-------------------+-----------------------+----------------------------+-------------+-------------------------+----------------------+-----------------+------------------+
| 292787210|2022-01-14 01:31:21|                   null|                         Web|  GoDaddy.com|                 ccavenue|                   Web|               MH|        My Account|
| 292787210|2022-02-17 06:49:11|                   null|                         Web|  GoDaddy.com|                 ccavenue|                   Web|               MH|        My Account|
| 292787210|2022-04-24 20:58:05|                   null|              

In [5]:
ds = tecton.get_workspace('jfarr').get_data_source('ads_bill_line_batch')
ds.get_dataframe(datetime(2021, 11, 10), datetime(2022, 8, 1)).to_spark().explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
Project (3)
+- Filter (2)
   +- Scan parquet tecton_dev.ads_bill_line (1)


(1) Scan parquet tecton_dev.ads_bill_line
Output [184]: [bill_id#1201, bill_line_num#1202, bill_country_code#1203, rep_version_id#1204, private_label_id#1205, bill_isc_source_code#1206, bill_sub_geo_code#1207, bill_postal_code#1208, bill_source_name#1209, entered_by_name#1210, translation_language_name#1211, primary_payment_type_name#1212, primary_payment_subtype_name#1213, secondary_payment_type_name#1214, secondary_payment_subtype_name#1215, bill_modified_mst_date_prior_year#1216, bill_isc_source_code_desc#1217, bill_isc_company_id#1218, bill_isc_campaign_id#1219, bill_isc_channel_id#1220, bill_isc_product_id#1221, bill_isc_vendor_id#1222, bill_isc_media_specialist_id#1223, bill_isc_sub_channel_id#1224, bill_isc_channel_name#1225, bill_isc_sub_channel_name#1226, bill_isc_vendor_name#1227, bill_isc_product_name#1228, bill_isc_media_specialist_name#1229, reseller_type_name#1230, reseller_nam

In [6]:
fv = tecton.get_workspace('jfarr').get_feature_view('customer_last_order_90d')
fv.get_historical_features(start_time=datetime(2022, 7, 10), end_time=datetime(2022, 8, 15), from_source=True).to_spark().explain(mode='formatted')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (9)
+- Project (8)
   +- Filter (7)
      +- Window (6)
         +- Sort (5)
            +- Exchange (4)
               +- Project (3)
                  +- Filter (2)
                     +- Scan parquet tecton_dev.ads_bill_line (1)


(1) Scan parquet tecton_dev.ads_bill_line
Output [10]: [bill_sub_geo_code#1760, primary_payment_type_name#1765, reseller_name#1784, c3_call_center_location_name#1791, merged_shopper_id#1799, crm_portfolio_type_name#1814, bill_modified_mst_ts#1841, purchase_path_name#1846, point_of_purchase_name#1932, partition_bill_mst_year_month#1937]
Batched: true
Location: InMemoryFileIndex [s3://gd-gdmltecton-stage-athena-queries/GD-AWS-USA-GD-gdmltecton-Ops/ads_bill_line]
PushedFilters: [IsNotNull(partition_bill_mst_year_month), GreaterThanOrEqual(partition_bill_mst_year_month,2022-07), LessThanOrEqual(partition_bill_mst_year_month,2022-08)]
ReadSchema: struct<bill_sub_geo_code:string,primary_payment_type_name:string,reseller_nam

### get_historical_features

In [4]:
import pandas
import tecton
from datetime import datetime

ws = tecton.get_workspace('jfarr')
fv = ws.get_feature_view('customer_last_order_90d')

spine = pandas.DataFrame({'shopper_id': ['292787210'], 'window_end_ts': datetime(2022, 3, 16, 23, 32, 57)})

df = fv.get_historical_features(spine, from_source=True).to_spark()
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [5]:
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+------------------------------------------------+-----------------------------------------------------+--------------------------------------+--------------------------------------------------+-----------------------------------------------+------------------------------------------+-------------------------------------------+
|shopper_id|window_end_ts      |customer_last_order_90d__crm_portfolio_type_name|customer_last_order_90d__c3_call_center_location_name|customer_last_order_90d__reseller_name|customer_last_order_90d__primary_payment_type_name|customer_last_order_90d__point_of_purchase_name|customer_last_order_90d__bill_sub_geo_code|customer_last_order_90d__purchase_path_name|
+----------+-------------------+------------------------------------------------+-----------------------------------------------------+--------------------------------------+--------------------------------------------------+-----------------------------------------------+-----